In [3]:
import requests
import pandas as pd
url = 'https://mediadive.dsmz.de/rest/media'

response = requests.get(url)

if response.status_code == 200:

    resp_dict = response.json()
    df = pd.DataFrame(resp_dict.get('data'))
    print(df)
    #df.to_csv('medialist.csv')
else:
    print(response.status_code)

       id                                               name  complex_medium  \
0       1                                      NUTRIENT AGAR               1   
1      1a                  REACTIVATION WITH LIQUID MEDIUM 1               1   
2       2                          BACILLUS PASTEURII MEDIUM               1   
3       3                                 AZOTOBACTER MEDIUM               0   
4       6                           ALLANTOIN MINERAL MEDIUM               1   
...   ...                                                ...             ...   
3318   P6                                              mTA10               1   
3319   P7            N27 RHODOSPIRILLACEAE MEDIUM (modified)               1   
3320   P8                                      M9-SNG medium               1   
3321   P9                                     PDB-SNG medium               1   
3322  P10  BekiSolidMedium (Hydrogenovibrio crunogenus QU...               1   

      source                           

In [39]:
from pandas import json_normalize
url = 'https://mediadive.dsmz.de/rest/medium-composition/119'

resp = requests.get(url).json()
if resp.get('status') == 200:
    print("Data fetched successfully")
    df = json_normalize(resp.get('data'))
    print(df)
else:
    print(f"Error: {resp.get('error')} - {resp.get('msg')}")


Data fetched successfully
                       name    id       g_l     mmol_l  optional
0                    NaHCO3    55  3.988040  47.472700         0
1                Na-formate   144  1.994020  29.320700         0
2                Na-acetate    23  0.997009  12.153600         0
3                    KH2PO4    11  0.498504   3.663180         0
4      L-Cysteine HCl x H2O   770  0.498504   2.838280         0
5              Na2S x 9 H2O    54  0.498504   2.075530         0
6                     NH4Cl    42  0.398804   7.455670         0
7                      NaCl    43  0.398804   6.824150         0
8             MgSO4 x 7 H2O     8  0.398804   1.618020         0
9             Yeast extract    16  0.199402        NaN         0
10            CaCl2 x 2 H2O     7  0.049850   0.339084         0
11                      HCl    68  0.002493   0.068363         0
12            FeSO4 x 7 H2O    12  0.001994   0.007172         0
13            FeCl2 x 4 H2O   260  0.001496   0.007522         0

In [53]:
url = 'https://mediadive.dsmz.de/rest/medium-composition/120'
df_2 = json_normalize(requests.get(url).json().get('data'))

ndf1=df[["name", "g_l"]]
ndf2=df_2[["name", "g_l"]]

merged_df = pd.merge(ndf1, ndf2, on="name", how="outer", suffixes=('_df1', '_df2'))
print(merged_df)

                          name   g_l_df1       g_l_df2
0       (DL)-alpha-Lipoic acid       NaN  4.882810e-05
1                       Biotin       NaN  1.953120e-05
2                CaCl2 x 2 H2O  0.049850  2.441410e-01
3   Calcium D-(+)-pantothenate       NaN  4.882810e-05
4                     Casitone       NaN  1.953120e+00
5                CoCl2 x 6 H2O  0.000189  1.855470e-04
6                CuCl2 x 2 H2O  0.000002  1.953130e-06
7      DL-2-Methylbutyric acid       NaN           NaN
8              Distilled water       NaN           NaN
9                FeCl2 x 4 H2O  0.001496  1.464840e-03
10               FeSO4 x 7 H2O  0.001994  1.953120e-03
11                  Folic acid       NaN  1.953120e-05
12                       H2SO4       NaN           NaN
13                       H3BO3  0.000006  5.859380e-06
14                         HCl  0.002493  2.441410e-03
15             Isobutyric acid       NaN           NaN
16             Isovaleric acid       NaN           NaN
17        

In [68]:
import requests
import pandas as pd
from pandas import json_normalize

merged_df = pd.DataFrame()

for num in [1,2,3,5,119,120]:
    url = f'https://mediadive.dsmz.de/rest/medium-composition/{num}'
    if requests.get(url).status_code==200: 
        df = json_normalize(requests.get(url).json().get('data'))[["name", "g_l"]]
        df = df.rename(columns={"g_l": f"{num}"})  # give each column a unique name
        if merged_df.empty:
            merged_df = df
        else:
            merged_df = pd.merge(merged_df, df, on="name", how="outer")
    else:
        print('error',requests.get(url).status_code, ' for ',num)


print(merged_df)

error 404  for  5
                          name     1     2       3       119           120
0       (DL)-alpha-Lipoic acid   NaN   NaN     NaN       NaN  4.882810e-05
1                         Agar  15.0  15.0  15.000       NaN           NaN
2                       Biotin   NaN   NaN     NaN       NaN  1.953120e-05
3                        CaCO3   NaN   NaN   5.000       NaN           NaN
4                CaCl2 x 2 H2O   NaN   NaN   0.100  0.049850  2.441410e-01
5   Calcium D-(+)-pantothenate   NaN   NaN     NaN       NaN  4.882810e-05
6                     Casitone   NaN   NaN     NaN       NaN  1.953120e+00
7                CoCl2 x 6 H2O   NaN   NaN     NaN  0.000189  1.855470e-04
8                CuCl2 x 2 H2O   NaN   NaN     NaN  0.000002  1.953130e-06
9      DL-2-Methylbutyric acid   NaN   NaN     NaN       NaN           NaN
10             Distilled water   NaN   NaN     NaN       NaN           NaN
11               FeCl2 x 4 H2O   NaN   NaN     NaN  0.001496  1.464840e-03
12     

In [ ]:
df_list=[]
for num in [382,383]:
    url = f'https://mediadive.dsmz.de/rest/medium-strains/{num}'
    if requests.get(url).status_code==200: 
        df = json_normalize(requests.get(url).json().get('data'))[["species","bacdive_id"]]
        df = df.rename(columns={"species": f"species_{num}","bacdive_id": f"id_{num}"})  # give each column a unique name
        df_list.append(df)
    else:
        print('error',requests.get(url).status_code, ' for ',num)
merged_df = pd.concat(df_list, axis=1)

print(merged_df)



              species_382   id_382                            species_383  \
0        Escherichia coli   4437.0                       Desulfocella sp.   
1        Escherichia coli   4453.0             Desulfobacterium indolicum   
2        Escherichia coli   4474.0                Desulfobacula phenolica   
3        Escherichia coli   4475.0                 unclassified bacterium   
4        Escherichia coli   4476.0                      Malonomonas rubra   
5        Escherichia coli   4477.0                 Desulfotignum balticum   
6        Escherichia coli   4559.0                   Desulfobacterium sp.   
7        Escherichia coli   4598.0                Desulfobacula toluolica   
8        Escherichia coli   4625.0                   Desulfobacterium sp.   
9        Escherichia coli   4626.0                      Desulfococcus sp.   
10       Escherichia coli   4882.0               Desulfofustis glycolicus   
11       Escherichia coli   4883.0                      Malonomonas rubra   

In [106]:
import bacdive

client = bacdive.BacdiveClient('tanliangxun2000@gmail.com', 'pass1234')

client.search(id='4437;4453')

for strain in client.retrieve():
    print(strain["General"])
for strain in client.retrieve(['NCBI tax id']):
    print(strain)
    print(strain[list(strain)[0]])
    print(strain[list(strain)[0]][0]['NCBI tax id']['NCBI tax id'])

-- Authentication successful --
{'@ref': 1390, 'BacDive-ID': 4453, 'DSM-Number': 3423, 'keywords': 'Bacteria', 'description': 'Escherichia coli K12 JM109 is a bacterium of the family Enterobacteriaceae.', 'NCBI tax id': {'NCBI tax id': 562, 'Matching level': 'species'}, 'strain history': {'@ref': 1390, 'history': '<- H. Wolf <- J. Messing'}, 'doi': '10.13145/bacdive4453.20241212.9.2'}
{'@ref': 1141, 'BacDive-ID': 4437, 'DSM-Number': 2829, 'keywords': ['Bacteria', 'antibiotic resistance'], 'description': 'Escherichia coli K12 JM103 is a bacterium that has multiple antibiotic resistances.', 'NCBI tax id': {'NCBI tax id': 562, 'Matching level': 'species'}, 'strain history': {'@ref': 1141, 'history': '<- M. Ranki'}, 'doi': '10.13145/bacdive4437.20241212.9.2'}
{'4453': [{'NCBI tax id': {'NCBI tax id': 562, 'Matching level': 'species'}}, {'NCBI tax id': 562}]}
[{'NCBI tax id': {'NCBI tax id': 562, 'Matching level': 'species'}}, {'NCBI tax id': 562}]
562
{'4437': [{'NCBI tax id': {'NCBI tax i

In [38]:
resp = requests.get(url).json()
data = resp.get('data')
if isinstance(data, dict):
    df = pd.DataFrame([data])  # Wrap the dictionary in a list
else:
    df = pd.DataFrame(data)
print(df)

   id     name  ChEBI   CAS-RN KEGG-Compound BRENDA-Ligand MetaCyc  PubChem  \
0   5  Glucose  17234  50-99-7        C00293          None    None     5793   

     ZVG  complex_compound    mass  formula  density  \
0  19010                 0  180.16  C6H12O6    1.544   

                                            synonyms roles  \
0  [(+)-Glucose, Anhydrous dextrose, Brake (pH-en...    []   

                                               media  
0  [3, 7, 8, 10, 11, 11b, 13, 21, 48, 53, 54, 58,...  


In [ ]:
url = 'https://mediadive.dsmz.de/rest/medium-strains/119'

resp = requests.get(url).json()
if resp.get('status') == 200:
    print("Data fetched successfully")
    df = json_normalize(resp.get('data'))
    print(df)
else:
    print(f"Error: {resp.get('error')} - {resp.get('msg')}")


Data fetched successfully
       id                                species        ccno  growth  \
0     501        Methanobrevibacter arboriphilus     DSM 744       1   
1     592             Methanobrevibacter smithii     DSM 861       1   
2     593                   Methanobacterium sp.     DSM 862       1   
3     594              Methanobacterium bryantii     DSM 863       1   
4     595              Methanospirillum hungatei     DSM 864       1   
..    ...                                    ...         ...     ...   
75  16235             Methanobacterium aggregans   DSM 29428       1   
76  43359            Methanobrevibacter millerae   DSM 16643       1   
77  49551  Methanobacterium alkalithermotolerans  DSM 102889       1   
78  47201                   Methanobacterium sp.   DSM 28100       1   
79  53291                Aceticella autotrophica  DSM 108286       1   

    bacdive_id domain  
0         6957      A  
1         6963      A  
2         6904      A  
3         690

In [21]:
url = 'https://mediadive.dsmz.de/rest/ingredient/5'

resp = requests.get(url).json()
if resp.get('status') == 200:
    print("Data fetched successfully")
    df = json_normalize(resp.get('data'))
    print(df)
else:
    print(f"Error: {resp.get('error')} - {resp.get('msg')}")


Data fetched successfully
   id     name  ChEBI   CAS-RN KEGG-Compound BRENDA-Ligand MetaCyc  PubChem  \
0   5  Glucose  17234  50-99-7        C00293          None    None     5793   

     ZVG  complex_compound    mass  formula  density  \
0  19010                 0  180.16  C6H12O6    1.544   

                                            synonyms roles  \
0  [(+)-Glucose, Anhydrous dextrose, Brake (pH-en...    []   

                                               media  
0  [3, 7, 8, 10, 11, 11b, 13, 21, 48, 53, 54, 58,...  


In [37]:
url = 'https://mediadive.dsmz.de/rest/ingredient/5'

resp = requests.get(url).json()
if resp.get('status') == 200:
    print("Data fetched successfully")
    allmediawithglucose_df = json_normalize(resp.get('data'))

dic1= {}
for medianum in allmediawithglucose_df['media'].iloc[0]:
    #print(medianum)
    resp_c = requests.get(f'https://mediadive.dsmz.de/rest/medium-composition/{medianum}').json()
    if resp_c.get('status') == 200:
        print("Data fetched successfully")
        mediacomposition_df = json_normalize(resp_c.get('data'))
        filtered_rows = mediacomposition_df[mediacomposition_df['id'].isin([5, 4])]
        for _, row in filtered_rows.iterrows():
            print(row['name'])
            print(row['g_l'])
            r_name= row['name']
            r_g_l= row['g_l']
            dic1[medianum] = {'name': r_name, 'g_l': r_g_l}
        print(dic1)
    else:
        print(f"Error: {resp_c.get('error')} - {resp_c.get('msg')}")


Data fetched successfully
Data fetched successfully
Glucose
5.0
Distilled water
nan
{3: {'name': 'Distilled water', 'g_l': nan}}
Data fetched successfully
Glucose
5.0
Distilled water
nan
{3: {'name': 'Distilled water', 'g_l': nan}}
Data fetched successfully
Glucose
1.0
Distilled water
nan
{3: {'name': 'Distilled water', 'g_l': nan}, 7: {'name': 'Distilled water', 'g_l': nan}}
Data fetched successfully
Glucose
1.0
Distilled water
nan
{3: {'name': 'Distilled water', 'g_l': nan}, 7: {'name': 'Distilled water', 'g_l': nan}}
Data fetched successfully
Glucose
5.0
Distilled water
nan
{3: {'name': 'Distilled water', 'g_l': nan}, 7: {'name': 'Distilled water', 'g_l': nan}, 8: {'name': 'Distilled water', 'g_l': nan}}
Data fetched successfully
Glucose
5.0
Distilled water
nan
{3: {'name': 'Distilled water', 'g_l': nan}, 7: {'name': 'Distilled water', 'g_l': nan}, 8: {'name': 'Distilled water', 'g_l': nan}}
Data fetched successfully
Glucose
20.0
Distilled water
nan
{3: {'name': 'Distilled water', '

KeyboardInterrupt: 

In [3]:
import pyrodigal

# Load the FASTA file
with open("./data/genomes/GCF_000006765.1_ASM676v1_genomic.fna") as f:
    sequence = "".join(line.strip() for line in f if not line.startswith(">"))

# Create a Pyrodigal gene finder object
gene_finder = pyrodigal.GeneFinder()

# Predict genes
predictions = gene_finder.find_genes(sequence)

# Output gene predictions
for gene in predictions:
    print(f"Gene {gene.locus_tag}:")
    print(f"  Start: {gene.start}")
    print(f"  End: {gene.stop}")
    print(f"  Strand: {'+' if gene.strand == 1 else '-'}")
    print(f"  Protein sequence:\n{gene.translation}")
    print()

RuntimeError: cannot find genes without having trained in single mode

In [4]:
import pyrodigal

# Initialize the annotator for multi-contig input
annotator = pyrodigal.Annotator()

# Read and annotate the FASTA file
with open("./data/genomes/GCF_000006765.1_ASM676v1_genomic.fna") as f:
    predictions = annotator.annotate_fasta(f)

# Output gene predictions
for gene in predictions:
    print(f"Gene {gene.locus_tag}:")
    print(f"  Contig: {gene.contig}")
    print(f"  Start: {gene.start}")
    print(f"  End: {gene.stop}")
    print(f"  Strand: {'+' if gene.strand == 1 else '-'}")
    print(f"  Protein sequence:\n{gene.translation}\n")

AttributeError: module 'pyrodigal' has no attribute 'Annotator'

In [7]:
pyrodigal

<module 'pyrodigal' from '/Users/liangxuntan/anaconda3/envs/mne/lib/python3.11/site-packages/pyrodigal/__init__.py'>

In [18]:
from Bio import SeqIO
import pyrodigal

# Initialize GeneFinder in metagenomic mode
gene_finder = pyrodigal.GeneFinder(meta=True)

# Iterate through each contig in the FASTA file
for record in SeqIO.parse("./data/genomes/GCF_000006765.1_ASM676v1_genomic.fna", "fasta"):
    print(f"\nContig: {record.id}")
    predictions = gene_finder.find_genes(str(record.seq))
    
    for gene in predictions:
        print(f"    Begin: {gene.begin}")
        print(f"    End: {gene.end}")
        print(f"    conf score: {gene.confidence()}")
        print(f"    AAseq: {gene.translate(translation_table=None, unknown_residue=88, include_stop=True, strict=True)}")
        print(f"    Strand: {'+' if gene.strand == 1 else '-'}")
        #print(f"    Translation:\n{gene.translation[:30]}...")  # Truncated


Contig: NC_002516.2
    Begin: 483
    End: 2027
    conf score: 99.99
    AAseq: MSVELWQQCVDLLRDELPSQQFNTWIRPLQVEAEGDELRVYAPNRFVLDWVNEKYLGRLLELLGERGEGQLPALSLLIGSKRSRTPRAAIVPSQTHVAPPPPVAPPPAPVQPVSAAPVVVPREELPPVTTAPSVSSDPYEPEEPSIDPLAAAMPAGAAPAVRTERNVQVEGALKHTSYLNRTFTFENFVEGKSNQLARAAAWQVADNLKHGYNPLFLYGGVGLGKTHLMHAVGNHLLKKNPNAKVVYLHSERFVADMVKALQLNAINEFKRFYRSVDALLIDDIQFFARKERSQEEFFHTFNALLEGGQQVILTSDRYPKEIEGLEERLKSRFGWGLTVAVEPPELETRVAILMKKAEQAKIELPHDAAFFIAQRIRSNVRELEGALKRVIAHSHFMGRPITIELIRESLKDLLALQDKLVSIDNIQRTVAEYYKIKISDLLSKRRSRSVARPRQVAMALSKELTNHSLPEIGVAFGGRDHTTVLHACRKIAQLRESDADIREDYKNLLRTLTT*
    Strand: +
    Begin: 2056
    End: 3159
    conf score: 99.99
    AAseq: MHFTIQREALLKPLQLVAGVVERRQTLPVLSNVLLVVEGQQLSLTGTDLEVELVGRVVLEDAAEPGEITVPARKLMDICKSLPNDVLIDIRVEEQKLLVKAGRSRFTLSTLPANDFPTVEEGPGSLNFSIAQSKLRRLIDRTSFAMAQQDVRYYLNGMLLEVNGGTLRSVATDGHRLAMCSLDAQIPSQDRHQVIVPRKGILELARLLTEQDGEVGIVLGQHHIRATTGEFTFTSKLVDGKFPDYERVLPRGGDKLVVGDRQQLREAFSRTAILSNEKYRGIRLQLSNGLLKIQANNPEQEEAEEEVQVEYNGGNLEIGFNVSY

In [8]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import pyrodigal

# Initialize Pyrodigal GeneFinder in metagenomic mode
gene_finder = pyrodigal.GeneFinder(meta=True)

# List to store predicted protein sequences
predicted_proteins = []

# Read each contig from the input FASTA file
for record in SeqIO.parse("/Users/liangxuntan/Code/fyp2025/data/labelled_genomes/562_GCF_000005845.2_ASM584v2_genomic.fna", "fasta"):
    predictions = gene_finder.find_genes(str(record.seq))
    
    for gene in predictions:
        if gene.confidence() >= 99.9999999:
            protein_seq = gene.translate()
            protein_id = f"{record.id}_gene_{gene.begin}_{gene.end}"
            predicted_proteins.append(SeqRecord(protein_seq, id=protein_id, description=""))

# Write predicted proteins to a FASTA file
SeqIO.write(predicted_proteins, "predicted_proteins.faa", "fasta")

print(f"Written {len(predicted_proteins)} protein sequences to 'predicted_proteins.faa'")


Written 1462 protein sequences to 'predicted_proteins.faa'


In [4]:
# --- Cell: count Pfam names in a *.pfamtblout file --------------------------
import collections, pandas as pd
from pathlib import Path

# ⇩ change to your actual path or leave as-is if the file is in the same folder
pfam_tbl_path = Path("efilt2_pfam_domains.pfamtblout")

# Counter for occurrences
pfam_counts = collections.Counter()

with pfam_tbl_path.open() as fh:
    for line in fh:
        # Skip comment lines that start with '#'
        if line.startswith("#") or not line.strip():
            continue
        fields = line.split()          # pfamtblout is space-delimited
        pfam_name = fields[0]          # 1stnd column = Pfam HMM/family name
        pfam_counts[pfam_name] += 1

# --------------- results ---------------------------------
print("Total unique Pfam families:", len(pfam_counts))
print("Example subset:", dict(list(pfam_counts.items())[:20]))

# convert to a DataFrame for sorting / plotting
pfam_df = (
    pd.Series(pfam_counts, name="count")
      .sort_values(ascending=False)
      .rename_axis("pfam_name")
      .reset_index()
)

pfam_df.head()      # display the top counts


Total unique Pfam families: 44
Example subset: {'Bac_DnaA': 1, 'Bac_DnaA_C': 1, 'DnaA_N': 1, 'DNA_pol3_beta': 1, 'DNA_pol3_beta_2': 1, 'DNA_pol3_beta_3': 1, 'SMC_N': 1, 'DNA_gyraseB': 1, 'GyrB_insert': 1, 'DNA_gyraseB_C': 1, 'Acyltransferase': 1, 'tRNA_synt_2f': 2, 'tRNA-synt_2e': 2, 'Adenine_glyco': 1, 'Lip_A_acyltrans': 1, 'Vut_1': 1, 'TrkA_N': 1, 'TrkA_C': 1, 'Methyltr_RsmB-F': 1, 'RsmF-B_ferredox': 1}


,pfam_name,count
0,tRNA_synt_2f,2
1,tRNA-synt_2e,2
2,Coprogen_oxidas,2
3,Trp_syntA,2
4,Sua5_yciO_yrdC,1
